# Exercise: Determing the Extent of Water Bodies

## Overview 
In this exercise, we will create a new notebook to determine the extent of water bodies using the Water Observation from Space Product(WOFS).The WOFs data will also be compared to the landsat 8 dataset.


The notebook will include the following steps:

* Load WOFS and Landsat 8 data
* Identify water pixels from WOFS
* Match landsat to WOFS and plot RGB
* Plot the total number of water pixels in each image, multiplied by the pixel area
* Customise the plot

At the conclusion of this exercise, you will have determine the extent of the selected water body in the selected area.

## Set up notebook

In your Training folder, create a new Python 3 notebook. Name it water_extent_exercise.ipynb. For more instructions on creating a new notebook, see the instructions from [Session 2](../session_2/04_load_data_exercise.ipynb).

### Load packages and functions

In the first cell, type the following code and then run the cell to import necessary Python dependencies.


    import matplotlib.pyplot as plt
    %matplotlib inline

    import sys
    import datacube

    sys.path.append('../Scripts')
    from deafrica_datahandling import load_ard, wofs_fuser
    from deafrica_plotting import display_map


### Connect to the datacube

Enter the following code and run the cell to create our dc object, which provides access to the datacube.

    dc = datacube.Datacube(app="water_extent_exercise")

In the next cell, enter the following code, and then run it to select an area.

#### Define the central point of interest
    lat = -6.0873
    lon = 35.1817

    lat_buffer = 0.2
    lon_buffer = 0.2

#### Combine central lat,lon with buffer to get area of interest

    lat_range = (lat - lat_buffer, lat + lat_buffer)
    lon_range = (lon - lon_buffer, lon + lon_buffer)

#### Define the start year and end year
    time = ('2018')

In the next cell, enter the following code, and then run it to show the area on a map. Since we have defined our area using the variables lon_range and lat_range, we can use those instead of typing out `(latitude-buffer, latitude+buffer)` and `(longitude-buffer, longitude+buffer)` again.

    display_map(x=lon_range, y=lat_range)

#### Query Object
    
Notice `lat_range`, `lon_range` and `time` were all defined in the previous cell, so we can use them as variables here
    
    query = {
        'x': lon_range,
        'y': lat_range,
        'resolution': (-30, 30),
        'output_crs':'EPSG:6933',
        'time': (time),
    }

#### Load Landsat Data

    ds_landsat = load_ard(dc=dc,
                         products=['ls8_usgs_sr_scene'],
                         measurements=['red', 'green', 'blue'],
                         group_by='solar_day',
                         **query)
                         
    ds_landsat


#### Load WOfS

    ds_wofs = dc.load(product=["ga_ls8c_wofs_2"],
             fuse_func=wofs_fuser,
             group_by='solar_day',
             **query
            )

    ds_wofs

#### Calculate the area per pixel
The number of pixels can be used for the area of the waterbody if the pixel area is known. 

    pixel_length = query["resolution"][1]  # in metres
    m_per_km = 1000  # conversion from metres to kilometres
    area_per_pixel = pixel_length**2 / m_per_km**2

#### select WOfS images at same timesteps as the Landsat data
    ds_wofs_landsat = ds_wofs.sel(time=ds_landsat.time, method='nearest')

#### remove any duplicate time-stamps
    index = np.unique(ds_wofs_landsat.time, return_index=True)
    ds_wofs_landsat = ds_wofs_landsat.isel(time=index[1])
    
    ds_wofs_landsat

#### Only show water pixels
    ds_valid_water = ds_wofs_landsat.water == 128

#### Calculate area of water pixels
    ds_valid_water_area = ds_valid_water.sum(dim=['x', 'y']) * area_per_pixel


#### Plot Monthly Time Series

    plt.figure(figsize=(18, 4))
    ds_valid_water_area.plot(marker='o', color='#9467bd')
    plt.title(f'Time Series Analysis of water observed area')
    plt.xlabel('Dates')
    plt.ylabel('Waterbody area (km$^2$)')
    plt.tight_layout()

    ds_wofs_landsat.isel(time=4).water.plot(cmap='Blues', robust=True)

    rgb(ds_landsat, index=4, bands=['red', 'green','blue'])

## Conclusion

Congratulations! You have successfully modified a notebook to create a quarterly geomedian composite by resampling Sentinel-2 data.

If you'd like to experiment futher, try running the code with different resampling values. Did you learn anything interesting to share with us?

In the next section, we will continue working with this notebook to calculate the NDVI values for each of our quarterly geomedians.